### Webscraper zum Erfassen der Berliner Immobiliendaten

[Immobilienrechner - Main](./immo_main.ipynb)<br>
[Immobilienrechner - Explorative Datenanalyse](./immo_eda.ipynb)<br>
[Immobilienrechner - Modellentwicklung](./immo_model.ipynb)<br>
[Immobilienrechner - Bereitstellung des besten Preisvorhersagemodells per Webinterface](./flask/immo_flask.ipynb)<br><br>
Das Einlesen der Daten erfolgt via Selenium mit Hilfe von Firefox.

Aufgrund rechtlicher Bestimmungen wurde die Url, welche durchsucht wird,
aus dem Quellcode entfernt und durch eine manuelle Eingabe ersetzt.

In [8]:
import numpy as np
import pandas as pd
import os
import sys
import validators
import shutil
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse
from datetime import datetime

#### Manuelle Url-Eingabe

In [9]:
while True:
    my_url = input('Zu "scrapende" Url eingeben (inklusive Protokoll):')
    if validators.url(my_url):
        break
parsed = urlparse(my_url)
base_url = parsed.scheme + '://' + parsed.netloc
print('Url2scrap:', my_url)
print('Base Url:', base_url)

Zu "scrapende" Url eingeben (inklusive Protokoll): https://www.ebay-kleinanzeigen.de/s-wohnung-kaufen/berlin/seite:1/c196l3331


Url2scrap: https://www.ebay-kleinanzeigen.de/s-wohnung-kaufen/berlin/seite:1/c196l3331
Base Url: https://www.ebay-kleinanzeigen.de


#### Webscrap-Routine

In [ ]:
result = []
max_visits = 1000
driver = webdriver.Firefox()
nan_count = 0
previous_found = 0

for page_nr in range(1, max_visits+1) :
    
    print('Lade Seite:', page_nr, my_url)
    sleep(randint(5,8))
    driver.get(my_url)
    
    li_items = driver.find_elements(By.XPATH, '//li[contains(@class,"ad-listitem lazyload-item")]')

    for item in li_items:

        artikel_name = item.find_element(By.XPATH, './/article')
        artikel_name = artikel_name.get_attribute('data-href').split('/')[2]

        try:
            plz_elm = item.find_element(By.XPATH, './/div[@class="aditem-main--top--left"]')
        except: 
            plz_elm = None
        if plz_elm:
            plz = plz_elm.text
            if plz == '': 
                #print('-------------- Alternative Plz !!!')
                htm_str = plz_elm.get_attribute('outerHTML')
                plz = htm_str[htm_str.find('i>')+2:htm_str.rfind('<')].strip()
        else:
            print('Kein Tag fuer Plz gefunden')

        try:
            preis_elm = item.find_element(By.XPATH, './/p[contains(@class,"price")]')
        except: 
            preis_elm = None
        if preis_elm:
            preis = preis_elm.text
            if preis == '': 
                #print('-------------- Alternativer Preis !!!')
                htm_str = preis_elm.get_attribute('outerHTML')
                preis = htm_str[htm_str.find('>')+1:htm_str.rfind('<')].strip()
        else:
            print('Kein Tag fuer Preis gefunden')

        try:
            ende = item.find_element(By.XPATH, './/p[@class="text-module-end"]')
        except:
            try:
                ende = item.find_element(By.XPATH, 
                        './/div[@class="aditem-main--bottom"]/p')
            except:
                ende = None

        if ende:
            spans = ende.find_elements(By.XPATH, './/span')
            if len(spans) > 0:
                groesse = spans[0].text
                if len(spans) > 1:
                    zimmer = spans[1].text
                else:
                    zimmer = 'Keine Zimmer gefunden'
            else:
                groesse = 'Keine Groessenangabe gefunden'
                zimmer = 'Keine Zimmer gefunden'
                htm = ende.get_attribute('innerHTML')
                htm = ende.get_attribute('outerHTML')
        else:
            print('Kein Tag fuer Ende gefunden')

        if groesse == '' and zimmer == '':
            groesse = spans[0].get_attribute('innerHTML').strip()
            zimmer = spans[1].get_attribute('innerHTML').strip()
        
        try:
            link_elm = item.find_element(By.XPATH, './/a[@class="ellipsis"]')
        except: 
            link_elm = None
        if link_elm:
            link = link_elm.get_attribute('href')
            if not link.startswith('http'):
                link = 'https://www.ebay-kleinanzeigen.de' + link
            if plz == '': 
                #print('-------------- Alternativer Link !!!')
                pass
        else:
            print('kein tag fuer link gefunden')
        #print('link:', link)
        
        # werte ausschneiden, pruefen und speichern
        if len(plz.split()) > 1:
            plz_new = plz.split()[0].strip()
            stadtteil = plz.split()[1].strip()
        else:
            plz_new = plz.split()[0].strip()
            stadtteil = np.nan
            nan_count += 1
        try:
            plz_new = int(plz_new)
        except:
            plz_new = np.nan
            nan_count += 1
        try:
            preis = int(preis.split()[0].strip().replace('.',''))
        except: 
            preis = np.nan
            nan_count += 1
        try:
            groesse = float(groesse.split()[0].strip().replace(',','.'))
        except:
            groesse = np.nan
            nan_count += 1
        try: 
            zimmer = float(zimmer.split()[0].strip().replace(',','.'))
        except: 
            zimmer = np.nan
            nan_count += 1
        result.append((plz_new, stadtteil, preis, groesse, zimmer, artikel_name, link))
    
    if len(result) == previous_found:
        print(f'FEHLER: Keine Ergebnisse auf Seite {my_url}')
    previous_found = len(result) 
    print('gefunden:', len(result), 'nan:', nan_count)
    
    try:
        next_url_elm = driver.find_element(By.XPATH, 
                            './/div[@class="pagination-nav"]/a[@class="pagination-next" and @title="Nächste"]')
    except: 
        next_url_elm = None
    if next_url_elm:
        next_url = next_url_elm.get_attribute('href')
        if not next_url.startswith('http'):
            print(next_url)
            next_url = base_url + next_url
            print(next_url)
    else:
        print('Kein Tag fuer den nächsten Link gefunden -> fertig')
        break
    print('Nächste Seite:', next_url)
    my_url = next_url
        
driver.quit()
print('Webscrap beendet')

#### DataFrame erstellen und als csv Datei speichern

In [12]:
df = pd.DataFrame(result, columns=['plz', 'stadtteil', 'preis', 'groesse',
                                'zimmer', 'a_name', 'link'])
f_name = 'eka__'+str(page_nr)+'_pages_'+str(len(result))+'_rows_'+ \
          str(nan_count)+'_nans__'+datetime.today().strftime('%Y-%m-%d %H:%M:')+'.csv'
df.to_csv(f_name, index=False)

if os.path.is_file('eka.csv'):
    if os.path.is_file('eka.csv.bak'): os.remove('eka.csv.bak')
    shutil.copy('eka.csv', 'eka.csv.bak')
shutil.copy(f_name, 'eka.csv')